<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Fruits_and_Vegetables_Image_Recognition_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kritikseth/fruit-and-vegetable-image-recognition

In [ ]:
import zipfile

In [ ]:
import tensorflow as tf


In [ ]:
with zipfile.ZipFile('/content/fruit-and-vegetable-image-recognition.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
test_path='/content/test'
train_path='/content/train'
valid_path='/content/validation'
img_size=(224,224)

train_data=tf.keras.preprocessing.image_dataset_from_directory(train_path,
                                                               image_size=img_size,
                                                               shuffle=True,
                                                               label_mode='categorical',
                                                               batch_size=32,
                                                               crop_to_aspect_ratio=True)

test_data=tf.keras.preprocessing.image_dataset_from_directory(test_path,
                                                               image_size=img_size,
                                                               shuffle=True,
                                                               label_mode='categorical',
                                                               batch_size=32,
                                                               crop_to_aspect_ratio=True)

valid_data=tf.keras.preprocessing.image_dataset_from_directory(valid_path,
                                                               image_size=img_size,
                                                               shuffle=True,
                                                               label_mode='categorical',
                                                               batch_size=32,
                                                               crop_to_aspect_ratio=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class_names=train_data.class_names

In [ ]:
plt.figure(figsize=(10,10))
for image,label in train_data.take(1):
  for i in range(12):
    plt.subplot(3,4,i+1)
    plt.imshow(image[i].numpy().astype('uint8'))
    plt.title(class_names[label[i].numpy().argmax()])
    plt.axis('off')

In [ ]:
# from functools import partial

# Conv2D=partial(tf.keras.layers.Conv2D,
#                kernel_size=7,
#                filters=32,
#                padding='same',
#                activation='relu',
#                kernel_initializer='he_normal',
#                strides=2
#                )

In [ ]:
n_classes = len(train_data.class_names)

In [ ]:
# model=tf.keras.Sequential([
#     tf.keras.layers.Input(shape=(224,224,3)),
#     tf.keras.layers.Rescaling(1/255.),
#     Conv2D(),
#     tf.keras.layers.MaxPool2D(),
#     Conv2D(filters=64,kernel_size=3),
#     Conv2D(filters=128, kernel_size=3),
#     tf.keras.layers.MaxPool2D(),
#     Conv2D(filters=256,kernel_size=3),
#     Conv2D(filters=512, kernel_size=3),
#     tf.keras.layers.MaxPool2D(),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
#     tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.Dense(n_classes, activation='softmax')


# ])

In [ ]:
# model.compile(loss='categorical_crossentropy',
#               optimizer = tf.keras.optimizers.Nadam(),
#               metrics=['accuracy'])

In [ ]:
# model.fit(train_data, validation_data=valid_data, epochs=3)

In [ ]:
inputs=tf.keras.layers.Input(shape=(224,224,3))

x=tf.keras.applications.xception.preprocess_input(inputs)

base_model=tf.keras.applications.Xception(weights='imagenet', include_top=False,input_tensor=x)
base_model.trainable=True
for layer in base_model.layers[100:]:
  layer.trainable=False

avg_pool = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

x=tf.keras.layers.Dense(128,activation='relu', kernel_initializer='he_normal')(avg_pool)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(n_classes, activation='softmax')(x)

model=tf.keras.Model(inputs,output)


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Nadam(),
              metrics=['accuracy'])

In [ ]:
model.fit(train_data, validation_data=valid_data, epochs=3)